In [1]:
import warnings
warnings.filterwarnings("ignore")

In [21]:
import numpy as np
import pandas as pd 
from sklearn.metrics import log_loss
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector
from sklearn.neighbors import KNeighborsRegressor
from sklearn.compose import make_column_transformer 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import KFold, GridSearchCV

In [10]:
housign_df = pd.read_csv(r"C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Data Sets\Boston.csv")
housign_df

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [11]:
X = housign_df.drop('medv', axis=1)
y = housign_df['medv']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=24, test_size=0.3)


In [32]:
X_train.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
64,0.01951,17.5,1.38,0,0.4161,7.104,59.5,9.2229,3,216,18.6,393.24,8.05
238,0.08244,30.0,4.93,0,0.4280,6.481,18.5,6.1899,6,300,16.6,379.41,6.36
94,0.04294,28.0,15.04,0,0.4640,6.249,77.3,3.6150,4,270,18.2,396.90,10.59
132,0.59005,0.0,21.89,0,0.6240,6.372,97.9,2.3274,4,437,21.2,385.76,11.12
366,3.69695,0.0,18.10,0,0.7180,4.963,91.4,1.7523,24,666,20.2,316.03,14.00


In [34]:
ohe = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')
scaler_mm = MinMaxScaler()
scaler_std = StandardScaler()

ct = make_column_transformer(
    ('passthrough', make_column_selector(dtype_exclude=object)),
    (ohe, make_column_selector(dtype_include=object)),
    verbose_feature_names_out=False
).set_output(transform='pandas')

knn = KNeighborsRegressor()

pipe = Pipeline(
    [
        ('CT', ct),
        ('SCL', None),
        ('KNN', knn)
    ]
)

# Changed to KFold instead of StratifiedKFold
kfold = KFold(n_splits=5, random_state=24, shuffle=True)

params = {
    'SCL':[scaler_mm, scaler_std, None],
    'KNN__n_neighbors': np.arange(1, 16),
    'KNN__metric': ['cityblock','haversine', 'manhattan', 'minkowski'] #Distance metrics
}

# Regression metrics for scoring
scoring_metrics = {
    'neg_mean_absolute_error': 'neg_mean_absolute_error',
    'neg_mean_squared_error': 'neg_mean_squared_error',
    'r2': 'r2'
}

gcv = GridSearchCV(
    pipe,
    param_grid=params,
    scoring=scoring_metrics, 
    cv=kfold, 
    verbose=2,
    refit='neg_mean_absolute_error'  # Choose which metric to use for best model selection
)

gcv.fit(X,y)

Fitting 5 folds for each of 180 candidates, totalling 900 fits
[CV] END KNN__metric=cityblock, KNN__n_neighbors=1, SCL=MinMaxScaler(); total time=   0.0s
[CV] END KNN__metric=cityblock, KNN__n_neighbors=1, SCL=MinMaxScaler(); total time=   0.0s
[CV] END KNN__metric=cityblock, KNN__n_neighbors=1, SCL=MinMaxScaler(); total time=   0.0s
[CV] END KNN__metric=cityblock, KNN__n_neighbors=1, SCL=MinMaxScaler(); total time=   0.0s
[CV] END KNN__metric=cityblock, KNN__n_neighbors=1, SCL=MinMaxScaler(); total time=   0.0s
[CV] END KNN__metric=cityblock, KNN__n_neighbors=1, SCL=StandardScaler(); total time=   0.0s
[CV] END KNN__metric=cityblock, KNN__n_neighbors=1, SCL=StandardScaler(); total time=   0.0s
[CV] END KNN__metric=cityblock, KNN__n_neighbors=1, SCL=StandardScaler(); total time=   0.0s
[CV] END KNN__metric=cityblock, KNN__n_neighbors=1, SCL=StandardScaler(); total time=   0.0s
[CV] END KNN__metric=cityblock, KNN__n_neighbors=1, SCL=StandardScaler(); total time=   0.0s
[CV] END KNN__met

GridSearchCV(cv=KFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000002AE83368EF0>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(drop='first',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000002AE83368FB0>)],
                                                          verbose_feature_names_out=False)),
                                       ('SCL', None),
                                       ('KNN', KNeighborsRegressor())]),
             param_grid={'KNN__metric': ['cityblock', 'haversine', 'manhattan',
                                         'minkowski'],
                         'KNN__n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]),
                         'SCL': [MinMaxScaler(), StandardScaler(), None]},
             scoring='neg_mean_absolute_error', verbose=2)

In [30]:
# Print results
print("\nBest Parameters:", gcv.best_params_)
print("\nBest Score:", gcv.best_score_)



Best Parameters: {'KNN__metric': 'cityblock', 'KNN__n_neighbors': 3, 'SCL': StandardScaler()}

Best Score: -2.7315220345563964


| Aspect | KFold | StratifiedKFold |
|--------|--------|-----------------|
| Purpose | General cross-validation, used for both regression and classification | Specifically designed for classification problems |
| Data Distribution | Splits data randomly without considering class distribution | Maintains the same class distribution in each fold as in the whole dataset |
| When to Use | • Regression problems<br>• Classification with balanced classes<br>• Large datasets | • Classification problems<br>• Imbalanced class distributions<br>• Small datasets |
| Advantages | • Simple implementation<br>• Works with any type of problem<br>• Faster computation | • Maintains class proportions<br>• Better for imbalanced datasets<br>• Reduces bias in evaluation |
| Disadvantages | • May create biased folds with imbalanced classes<br>• Less reliable for small imbalanced datasets | • Only works with classification<br>• Slightly more computational overhead |


In [31]:
# Print scores for all metrics
for metric in scoring_metrics:
    scores = pd.DataFrame(gcv.cv_results_)[f'mean_test_{metric}']
    print(f"\nBest {metric}: {scores.max()}")


Best neg_mean_absolute_error: -2.7315220345563964

Best neg_mean_squared_error: -17.64196869539895

Best r2: 0.7875954797972841
